In [1]:
%load_ext autoreload
%autoreload 2

from IPython.display import Image
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
import json
import numpy as np
import jax
import pickle
import matplotlib.pyplot as plt
import pandas as pd
from timecast.learners import AR
from timecast.learners._ar import _ar_predict, _ar_batch_window
from timecast.utils.numpy import ecdf
from timecast.utils.losses import MeanSquareError
import torch
import matplotlib

plt.rcParams['figure.figsize'] = [20, 10]

import tqdm.notebook as tqdm

/home/dsuo/miniconda3/envs/toy_flood/lib/python3.7/site-packages/jax/lib/xla_bridge.py:123: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [2]:
results = {}
for i in tqdm.tqdm([1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]):
    for j in [-8, -7, -6, -5, -4, -3, -2, -1]:
        name = "{}_{}".format(i, 10 ** j)
        with open("../tmp/batch_{}.pkl".format(name), "rb") as f:
            results[name] = pickle.load(f)

In [3]:
def rolling_mean(x, n):
    return pd.Series(x).rolling(window=n, min_periods=1).mean()

In [4]:
rolling_means = {}
for i, result in results.items():
    rolling_means[i] = rolling_mean([res["mse"].item() for res in result.values()], 531)

In [5]:
rolling_means = pd.DataFrame.from_dict(rolling_means)

In [6]:
from ealstm.gaip import FloodLSTM
from ealstm.gaip import FloodData
from ealstm.gaip.utils import MSE, NSE

from timecast.optim import SGD, Adam
from timecast.learners import AR

cfg_path = "/home/dsuo/src/toy_flood/ealstm/runs/run_2503_0429_seed283956/cfg.json"
ea_data = pickle.load(open("../ealstm/runs/run_2503_0429_seed283956/lstm_seed283956.p", "rb"))
flood_data = FloodData(cfg_path)

LR_AR = 1e-4
AR_INPUT_DIM=32
AR_OUTPUT_DIM=1

In [9]:
results = {}
mses = []
nses = []
for X, _, basin in tqdm.tqdm(flood_data.generator(), total=len(flood_data.basins)):
#     if basin == "01466500":
#         break
#     sgd = SGD(learning_rate=LR_AR, online=False)
    adam = Adam(learning_rate=lr)
    ar = AR(input_dim=AR_INPUT_DIM,
            output_dim=AR_OUTPUT_DIM,
            window_size=flood_data.cfg["seq_length"],
            optimizer=adam,
            history=X[:flood_data.cfg["seq_length"]],
            fit_intercept=True,
            constrain=False
           )

    # NOTE: difference in indexing convention, so need to pad one row
    X = np.vstack((X[flood_data.cfg["seq_length"]:], np.ones((1, X.shape[1]))))
    Y = np.array(ea_data[basin].qobs).reshape(-1, 1)

    Y_lstm = np.array(ea_data[basin].qsim).reshape(-1, 1)
    Y_target = Y - Y_lstm

    Y_ar = ar.predict_and_update(X, Y_target, batch_size)

    Y_hat = Y_lstm + Y_ar

    mse = MSE(Y, Y_hat)
    nse = NSE(Y, Y_hat)
    results[basin] = {
        "mse": mse,
        "nse": nse,
        "count": X.shape[0],
        "avg_mse": np.mean(np.array(mses)),
        "avg_nse": np.mean(np.array(nses))
    }
    mses.append(mse)
    nses.append(nse)
    print(basin, mse, nse, np.mean(np.array(mses)), np.mean(np.array(nses)))

In [46]:
for i in [1, 2, 4, 8, 16, 32, 64, 128]

96